In [1]:
!pip install tensorflow==2.1

     |████████████████████████████████| 421.8MB 31kB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 450kB 42.3MB/s 
     |████████████████████████████████| 3.9MB 22.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=f0c167982525747d5b0688bd050cdffa2c58e29cfc01dc55def9a45015fddf7d
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

In [2]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.1.0


In [5]:
users =['Mahsa', 'Reza','Nooshin','Ali']
movies = ['Star Wars', 'The Dark Knight', 'Shrek', 'The Incredibles', 'Bleu', 'Memento']
features = ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']

num_users = len(users)
num_movies = len(movies)
num_feats = len(features)
num_recommendations = 2

In [6]:
# each row represents a user's rating for the different movies
users_movies = tf.constant([
                [4,  6,  8,  0, 0, 0],
                [0,  0, 10,  0, 8, 3],
                [0,  6,  0,  0, 3, 7],
                [10, 9,  0,  5, 0, 2]],dtype=tf.float32)

# features of the movies one-hot encoded
# e.g. columns could represent ['Action', 'Sci-Fi', 'Comedy', 'Cartoon', 'Drama']
movies_feats = tf.constant([
                [1, 1, 0, 0, 1],
                [1, 1, 0, 0, 0],
                [0, 0, 1, 1, 0],
                [1, 0, 1, 1, 0],
                [0, 0, 0, 0, 1],
                [1, 0, 0, 0, 1]],dtype=tf.float32)

In [7]:
users_feats = tf.matmul(users_movies,movies_feats)
users_feats

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[10., 10.,  8.,  8.,  4.],
       [ 3.,  0., 10., 10., 11.],
       [13.,  6.,  0.,  0., 10.],
       [26., 19.,  5.,  5., 12.]], dtype=float32)>

In [8]:
users_feats = users_feats/tf.reduce_sum(users_feats,axis=1,keepdims=True)
users_feats

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[0.25      , 0.25      , 0.2       , 0.2       , 0.1       ],
       [0.0882353 , 0.        , 0.29411766, 0.29411766, 0.32352942],
       [0.44827586, 0.20689656, 0.        , 0.        , 0.3448276 ],
       [0.3880597 , 0.2835821 , 0.07462686, 0.07462686, 0.17910448]],
      dtype=float32)>

In [9]:
###Ranking feature relevance for each user
###We can use the users_feats computed above to represent the relative importance of each movie category for each user.

top_users_features = tf.nn.top_k(users_feats, num_feats)[1]
top_users_features

<tf.Tensor: shape=(4, 5), dtype=int32, numpy=
array([[0, 1, 2, 3, 4],
       [4, 2, 3, 0, 1],
       [0, 4, 1, 2, 3],
       [0, 1, 4, 2, 3]], dtype=int32)>

In [10]:
users_ratings = tf.matmul(users_feats,tf.transpose(movies_feats))
users_ratings

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[0.6       , 0.5       , 0.4       , 0.65      , 0.1       ,
        0.35      ],
       [0.4117647 , 0.0882353 , 0.5882353 , 0.67647064, 0.32352942,
        0.4117647 ],
       [1.        , 0.6551724 , 0.        , 0.44827586, 0.3448276 ,
        0.79310346],
       [0.8507463 , 0.6716418 , 0.14925373, 0.53731346, 0.17910448,
        0.5671642 ]], dtype=float32)>

In [64]:
loc = tf.where(users_movies==0)
users_ratings_new = np.zeros((num_users,num_movies))
users_ratings =np.array(users_ratings)
[i,j]=loc.shape
for i1 in range (i):
  l = np.array(loc[i1])
  users_ratings_new[l[0],l[1]] = users_ratings[l[0],l[1]]
users_ratings_new  

for j1 in range (num_users):
  N = np.where(users_ratings_new[j1,:] == max(users_ratings_new[j1,:]))
  print("User named {k} likes movie {n} more than others".format(k=users[j1],n=movies[int(N[0])]))

User named Mahsa likes movie The Incredibles more than others
User named Reza likes movie The Incredibles more than others
User named Nooshin likes movie Star Wars more than others
User named Ali likes movie Bleu more than others
